In [8]:
import os
import glob
import re
import numpy as np
import pandas as pd
import rasterio
from netCDF4 import Dataset
from datetime import datetime, timedelta
import geopandas as gpd
from shapely.geometry import Point

# 0. DIRECTORIES and PARAMETERS
glm_dir   = r'D:\1Research\2025\NOAA_SatHack\data\glm_ca_subset'
dry_tif   = r'D:\1Research\2025\NOAA_SatHack\data\sentinel_ca\sentinel_flammability_0p1deg.tif'

# shapefile folder and path for CA counties
shp_dir   = r'D:\1Research\2025\NOAA_SatHack\cb_2018_us_county_500k'
shp_path  = os.path.join(shp_dir, 'cb_2018_us_county_500k.shp')

# output Excel
out_xlsx  = r'D:\1Research\2025\NOAA_SatHack\figures\CA_county_DSI_summary.xlsx'

# geographic bounds and resolution 
lon_min, lon_max = -124.5, -114.0
lat_min, lat_max =   32.0,    42.0
res = 0.1

# 1. LOAD THE DRYNESS RASTER
with rasterio.open(dry_tif) as src:
    dryness = src.read(1)
nlat, nlon = dryness.shape

# build cell‐center coordinates
lon_edges = np.linspace(lon_min, lon_min + nlon * res, nlon + 1)
lat_edges = np.linspace(lat_min, lat_min + nlat * res, nlat + 1)
lon_centers = (lon_edges[:-1] + lon_edges[1:]) / 2
lat_centers = (lat_edges[:-1] + lat_edges[1:]) / 2
lons, lats = np.meshgrid(lon_centers, lat_centers)

# 2. BUILD HOURLY FLASH COUNTS
pattern = re.compile(r'_c(\d{4})(\d{3})(\d{2})(\d{2})(\d{2})')
counts_hour = {}

for nc in glob.glob(os.path.join(glm_dir, '*.nc')):
    fn = os.path.basename(nc)
    m = pattern.search(fn)
    if not m:
        continue
    year, doy, hr, mn, sc = map(int, m.groups())
    dt = datetime(year, 1, 1) + timedelta(days=doy-1,
                                         hours=hr,
                                         minutes=mn,
                                         seconds=sc)
    hour_key = dt.strftime('%Y%m%d%H')

    if hour_key not in counts_hour:
        counts_hour[hour_key] = np.zeros((nlat, nlon), dtype=float)

    with Dataset(nc, 'r') as ds:
        lat = ds.variables['flash_lat'][:]
        lon = ds.variables['flash_lon'][:]

    mask = (
        (lon >= lon_min)  & (lon < lon_max) &
        (lat >= lat_min)  & (lat < lat_max)
    )
    lon_f = lon[mask]
    lat_f = lat[mask]
    if lon_f.size == 0:
        continue

    ix = np.digitize(lon_f, lon_edges) - 1
    iy = np.digitize(lat_f, lat_edges) - 1

    grid = counts_hour[hour_key]
    for x, y in zip(ix, iy):
        if 0 <= x < nlon and 0 <= y < nlat:
            grid[y, x] += 1

# 3. LOAD CA COUNTIES and PREPARE MASKS
counties = gpd.read_file(shp_path)
ca = counties[counties.STATEFP == '06'].copy()
ca = ca.reset_index(drop=True)

# for each county build a boolean mask of grid‐cells whose center lies within it
county_masks = []
for geom in ca.geometry:
    pts = [Point(x, y) for x, y in zip(lons.ravel(), lats.ravel())]
    contains = np.array([geom.contains(pt) for pt in pts])
    county_masks.append(contains.reshape(nlat, nlon))

# 4. EXTRACT and AVERAGE DSI PER COUNTY
# list of hours 
hours = sorted(counts_hour.keys())
n_hours = len(hours)

# compute DSI time‐series per county
county_dsi_means = np.zeros((len(ca), n_hours), dtype=float)

for t_idx, hr in enumerate(hours):
    flash_grid = counts_hour[hr]
    dsi = flash_grid * dryness  # grid of DSI
    for j, mask in enumerate(county_masks):
        # mean over all cells in county 
        vals = dsi[mask]
        county_dsi_means[j, t_idx] = vals.mean() if vals.size else 0.0

# average over time
county_dsi_avg = county_dsi_means.mean(axis=1)

# assemble DataFrame
df = pd.DataFrame({
    'Name of county': ca.NAME,  
    'DSI values':     county_dsi_avg
})

# 5. SAVE TO EXCEL
df.to_excel(out_xlsx, index=False)
print(f"Wrote {len(df)} counties → {out_xlsx}")


Wrote 58 counties → D:\1Research\2025\NOAA_SatHack\figures\CA_county_DSI_summary.xlsx
